In [60]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread, Float
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap

# Load the Cornell Box
Thread.thread().file_resolver().append('cbox/')

scene = load_file('cbox/cbox_path.xml')
# scene = load_file('cbox/cbox_direct.xml')
# scene = load_file('cbox/cbox_path_tc.xml')
# scene = load_file('cbox/cbox_direct_tc.xml')
# scene = load_file('cbox/cbox_path_nee.xml')

# Find differentiable scene parameters
params = traverse(scene)

wall_res = params['wall_bsdf.reflectance.resolution']
wall_data = Float(params['wall_bsdf.reflectance.data'])
print(wall_res, wall_data)

params.keep(['wall_bsdf.reflectance.data'])

2021-11-29 17:13:32 INFO main [xml.cpp:1221] Loading XML file "cbox\cbox_path.xml" ..

2021-11-29 17:13:32 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-11-29 17:13:32 INFO main [xml.cpp:355] "cbox\cbox_path.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-11-29 17:13:32 INFO main [Scene] Building scene in OptiX ..

[16, 16] [0, 0, 0, 0, 0, .. 758 skipped .., 0.520996, 0.799103, 0.577581, 0.520996, 0.799103]


In [61]:
image_ref = render(scene, spp=512)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_ref.png', image_ref, crop_size)

![out_ref.png](out/out_ref.png)

In [62]:
params['wall_bsdf.reflectance.data'] = ek.full(Float, 1.0, len(wall_data))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=0.1)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_start.png', image, crop_size)
write_bitmap('out/wall_start.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

for it in range(200):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(wall_data - params['wall_bsdf.reflectance.data']))

    print('%g,%g' % (ob_val[0], err_ref[0]))

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_end.png', image, crop_size)
write_bitmap('out/wall_end.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

0.0168349,290.257
0.0179575,231.979
0.0224636,184.424
0.0157923,147.289
0.0160544,119.43
0.0155304,99.0561
0.0148055,84.6381
0.0203334,74.5318
0.0156634,67.5796
0.011891,62.5287
0.0156014,58.5737
0.0158082,55.1208
0.0146676,51.9302
0.0164926,48.836
0.0158336,45.7656
0.0150781,42.8129
0.0107285,39.9748
0.0169006,37.2657
0.0151311,34.8024
0.0178958,32.5851
0.0157934,30.5487
0.0154621,28.6969
0.0127597,26.9761
0.0183218,25.3378
0.0155182,23.7021
0.0135912,22.0908
0.0147128,20.4267
0.0142123,18.8076
0.0161875,17.26
0.0169142,15.9019
0.0183964,14.7182
0.0165211,13.6785
0.0191703,12.7813
0.0167789,12.053
0.0129358,11.4532
0.0169865,10.9606
0.0137617,10.5474
0.0204861,10.1864
0.0153403,9.83944
0.0134063,9.49581
0.0180794,9.12998
0.0133941,8.73067
0.0137775,8.31378
0.0168912,7.91555
0.0165986,7.52738
0.012853,7.15778
0.0152373,6.84917
0.0134901,6.5671
0.0128283,6.34817
0.0105722,6.14981
0.0181187,5.95741
0.0152482,5.79406
0.0130837,5.63967
0.01513,5.48831
0.0207371,5.337
0.0146551,5.24894
0.01

![start](out/out_start.png)
![end](out/out_end.png)
![start](out/wall_start.png)
![end](out/wall_end.png)